In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

In [2]:
df = pd.read_csv('/content/car_evaluation.csv')

In [3]:
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [4]:
df.columns = ['buying', 'maintenance', 'doors', 'persons', 'lug_boot', 'safety', 'class']

#df.rename({'vhigh':'buying'})

In [5]:
df.head()

,buying,maintenance,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [6]:
df['class'].value_counts()

unacc    1209
acc       384
good       69
vgood      65
Name: class, dtype: int64

In [7]:
#SMOTE - DE CAUTAT

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   buying       1727 non-null   object
 1   maintenance  1727 non-null   object
 2   doors        1727 non-null   object
 3   persons      1727 non-null   object
 4   lug_boot     1727 non-null   object
 5   safety       1727 non-null   object
 6   class        1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [16]:
df['doors'].unique()

array([2, 3, 4, 5])

In [12]:
df['doors'] = df['doors'].str.replace('more', '').astype(int)

In [15]:
df['persons'].unique()

array([2, 4, 5])

In [14]:
df['persons'] = df['persons'].str.replace('more', '5').astype(int)

In [17]:
!pip install category_encoders --quiet
import category_encoders as ce

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.8 MB/s eta 0:00:00


In [18]:
encoder = ce.OrdinalEncoder(cols = ['buying', 'maintenance', 'lug_boot', 'safety', 'class'])

In [19]:
df_enc = encoder.fit_transform(df)

In [20]:
df_enc.head()

,buying,maintenance,doors,persons,lug_boot,safety,class
0,1,1,2,2,1,1,1
1,1,1,2,2,1,2,1
2,1,1,2,2,2,3,1
3,1,1,2,2,2,1,1
4,1,1,2,2,2,2,1


In [21]:
df_enc['class'].unique()

array([1, 2, 3, 4])

In [22]:
X = df_enc.drop(columns = 'class')
y = df_enc['class']

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [25]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

In [37]:
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
gbc = GradientBoostingClassifier()

In [28]:
rf.fit(X_train, y_train)
et.fit(X_train, y_train)
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [29]:
y_pred_rf = rf.predict(X_test)
y_pred_et = et.predict(X_test)
y_pred_gbc = gbc.predict(X_test)

In [35]:
print('Random Forest: ',classification_report(y_test, y_pred_rf))
print('Extra Trees: ',classification_report(y_test, y_pred_et))
print('Gradinent Boosting: ',classification_report(y_test, y_pred_gbc))

Random Forest:                precision    recall  f1-score   support

           1       1.00      1.00      1.00       237
           2       0.95      0.97      0.96        77
           3       0.78      0.82      0.80        17
           4       0.91      0.67      0.77        15

    accuracy                           0.97       346
   macro avg       0.91      0.87      0.88       346
weighted avg       0.97      0.97      0.97       346

Extra Trees:                precision    recall  f1-score   support

           1       1.00      1.00      1.00       237
           2       0.96      0.96      0.96        77
           3       0.71      0.88      0.79        17
           4       0.83      0.67      0.74        15

    accuracy                           0.97       346
   macro avg       0.88      0.88      0.87       346
weighted avg       0.97      0.97      0.97       346

Gradinent Boosting:                precision    recall  f1-score   support

           1       1.00 

In [36]:
param_grid = {
    'n_estimators':[200, 500],
    'max_features':['auto', 'sqrt', 'log2'],
    'max_depth':[4, 5, 6, 8],
    'criterion':['gini', 'entropy']
}

In [38]:
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
gbc = GradientBoostingClassifier()

In [39]:
from sklearn.model_selection import GridSearchCV

cv_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5)

cv_rf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'sqrt',
 'n_estimators': 200}

In [40]:
best_params = cv_rf.best_params_

In [41]:
rf_cv = RandomForestClassifier(**best_params)

In [42]:
rf_cv.fit(X_train, y_train)

y_pred_rfcv = rf_cv.predict(X_test)

In [43]:
print(classification_report(y_test, y_pred_rfcv))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       237
           2       0.93      0.96      0.94        77
           3       0.63      0.71      0.67        17
           4       0.82      0.60      0.69        15

    accuracy                           0.96       346
   macro avg       0.84      0.82      0.82       346
weighted avg       0.96      0.96      0.96       346

